# Python Boot Camp


Welcome! 😃👋

In this notebook, we will go through some basic image processing in Python, come across standard tasks required while setting up deep learning pipelines, and familiarize ourselves with popular packages such as `glob`, `tifffile`, `tqdm`, `imgaug` and more.

We will be using sample images from the *MoNuSeg* dataset provided by [Kumar et al, 2018](https://ieeexplore.ieee.org/document/8880654). The data was publicly made available [here](https://monuseg.grand-challenge.org/) by the authors of the publication.

This dataset shows Hematoxylin and Eosin (H&E) Stained Images showing nuclei in different shapes.


***

## Table of contents:

0. Chapter - 0
  * [Downloading Data from an External URL](#zeroth)
1. Chapter - 1
  * [Images as Arrays](#first)   
  * [Image Channels](#second)
  * [Image Data Types](#third)
  * [Reshaping Images](#fourth)
  * [Normalizing Images](#fifth)
  * [Loading a Set of Images](#sixth)
2. Chapter - 2
  * [Cropping](#seventh)
  * [Downsampling](#eighth)
  * [Flipping](#ninth)
3. Chapter - 3
  * [Creating Batches](#tenth)
  * [Convolutions](#eleventh)
  * [Design your own filter](#twelvth)
4. Optional
  * [Data augmentation](#thirteenth)
***

### 0. Downloading data from an external url <a class="anchor" name="zeroth"></a>

Let us first download the images from an external url.
To do so, we need to import some commonly used dependencies.

In [ ]:
from pathlib import Path
import urllib.request, zipfile

Here, below is a helper function to download the data from an external url specified by argument `zip_url` and save it to a local directory specified by argument `project_name`. Let's execute the function (No output expected yet!).

In [ ]:
def extract_data(zip_url, project_name):
  zip_path = Path(project_name + '.zip')
  if (zip_path.exists()):
      print("Zip file was downloaded and extracted before!")
  else:
      urllib.request.urlretrieve(zip_url, zip_path)
      print("Downloaded data as {}".format(zip_path))
      with zipfile.ZipFile(zip_path, 'r') as zip_ref:
          zip_ref.extractall('./')
      print("Unzipped data to {}".format(Path(project_name)))


Now we call the function `extract_data` specifying desirable values of the arguments.

In [ ]:
extract_data(
    zip_url='https://owncloud.mpi-cbg.de/index.php/s/xwYonC9LucjLsY6/download',
    project_name = 'monuseg-2018',
)

In [ ]:
#!rm -rf monuseg-2018 monuseg-2018.zip

**Task**: Click on the `Files` directory (left panel) and check if some images exist within the `monuseg-2018` directory.

**Task**: Can you programmatically count the number of images and masks present in the `download/images` directory (Replace <path> with actual value).

*Hint*: Use `!ls -l <path> | wc -l`

## 1.1 Images as arrays <a class="anchor" name="first"></a>

2D Images are often represented as numpy arrays of shape (`height`, `width`, `num_channels`).

![RGB image as a numpy array](https://github.com/dlmbl/boot/assets/34229641/ce1ad3f3-dc34-46d1-b301-198768fbc369)

<div style="text-align: right"> Credit: <a href="https://e2eml.school/convert_rgb_to_grayscale.html">Brandon Rohrer’s Blog</a></div>


Multiple utilities/packages exist to read images from files in Python.
For example, one can use `tifffile.imread` to read `*.tif` images. <br>Another good package is `skimage.io.imread`.


If you look in the directory (`monuseg-2018/download`), you can see directories called `images` and `masks`.

Let's load one image and visualize it using `matplotlib.pyplot.imshow`. <br>
`matplotlib.pyplot.imshow` is the standard way to show images in jupyter notebooks!

In [ ]:
from tifffile import imread
from matplotlib.pyplot import imshow

img = imread('monuseg-2018/download/images/TCGA-2Z-A9J9-01A-01-TS1.tif')
print(f"Image `img` has type {type(img)}") # variable type
imshow(img)


**Task**:
Can you visualize the corresponding `mask` for the image above. <br>
(*Hint*: Look for the same name within the `masks` directory.) <br>
What does the mask show?



In [ ]:
mask = # TODO
print(f"Mask `mask` has type {type(mask)}") # variable type
imshow(mask)

## 1.2 Image channels <a class="anchor" name="second"></a>
If the image is a `grayscale` image, then the number of channels is equal to $1$,
in which case the array can also be of shape (height, width). <br>
If the image is `RGB`, then the number of channels is $3$.
with each channel encoding the red, green and blue components.


**Task**: Is <code>img</code> RGB or grayscale ?

*Hint*: <a href="https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Numpy_Python_Cheat_Sheet.pdf">numpy cheatsheet</a>

## 1.3 Image data types <a class="anchor" name="third"></a>


Images can be represented by a variety of data types. The following is a list of the most common datatypes:
- `bool`: binary, 0 or 1
- `uint8`: unsigned integers, 0 to 255 range
- `float`: -1 to 1 or 0 to 1


**Task**: What is the data type of <code> img</code>? What are the minimum and maximum intensity values?

*Hint*: <a href="https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Numpy_Python_Cheat_Sheet.pdf">numpy cheatsheet</a></div>

## 1.4 Reshaping Images <a class="anchor" name="fourth"></a>

`PyTorch`, `TensorFlow` and `JAX` are popular deep learning frameworks.
<br> In `PyTorch` images are represented as (`num_channels`, `height`, `width`).

But the image which we are working with has the `channel` as the last axis.

**Task**: Reshape <code>img</code> such that its shape is <code>(num_channels, height, width)</code>

*Hint*: <a href="https://numpy.org/doc/stable/reference/generated/numpy.transpose.html">numpy transpose</a>

In [ ]:
import numpy as np
img_reshaped = ## TODO
print(f"After reshaping, image has shape {img_reshaped.shape}")

# 1.5 Normalizing Images <a class="anchor" name="fifth"></a>

It often helps model training, if we provide image inputs to the model which are between [0,1] intensities. <br>
One way of normalizing an image is to divide the intensity on each pixel by the maximum allowed intensity for the available data type.

**Task**: Obtain an intensity normalized image using the idea above.


In [ ]:
def normalize(img):
  ###
  ### TODO: ADD CODE
  ###
  return

**Task**: What is the data type of the normalized image. Has it changed from before? Why?

## 1.6 Loading a set of images <a class="anchor" name="sixth"></a>

Given a set of images in a folder, we need to be able to easily find the pathnames and load them in. <br>
`glob` is a standard package that provides a utility for finding all pathnames that match a given pattern.

Here, our images have the `.tif` extension.

In [ ]:
import os
from glob import glob

img_dir = 'monuseg-2018/download/images/'
img_filenames = sorted(glob(os.path.join(img_dir, '*.tif')))

print(f"Found:")
for img_filename in img_filenames:
    print(f"{img_filename}")

**Task**: Load the set of masks, by correctly specifying the value of the variable `mask_filenames`

In [ ]:
#mask_dir = # TODO: fill value!!
#mask_filenames = # TODO: fill value!!
#for mask_filename in mask_filenames:
#   print(f"{mask_filename}")

Let's visualize some of the images and the corresponding mask, side by side. First let's provide a helper `visualize` function which takes two images as argument.


In [ ]:
from matplotlib.pyplot import subplot, figure, tight_layout, axis
import numpy
def visualize(im1, im2):
  figure(figsize=(10, 10))
  subplot(121)
  imshow(im1)
  axis('off')
  subplot(122)
  imshow(im2)
  axis('off')
  tight_layout()

Executing the cell below, would visualize a new random image and the corresponding segmentation mask, each time. This is because the variable `idx` gets a new value between $0$ and $14$ (there are $15$ images).

In [ ]:
idx = numpy.random.randint(len(img_filenames))
visualize(imread(img_filenames[idx]), imread(mask_filenames[idx]))


<hr>
Great Job! 🎊

**Checkpoint 1**

In the first chapter, we learnt about:

<li> image data types</li>
<li> reshaping images </li>
<li> normalizing images </li>
<li> Using <code>glob</code> to load a set of images
<hr>

**Bonus Task for Chapter 1**: Can you think of alternate approaches to intensity normalization? Any benefits of following one over the other?

## 2.1 Cropping <a class="anchor" name="seventh"></a>

While training models, we usually feed in smaller crops extracted from the original images.
To do so, we can rely on the powerful numpy [indexing](https://numpy.org/doc/stable/reference/arrays.indexing.html).

For example, let's extract the top left corner from one of our images.

In the cell below, the original image is visualized on the left and the cropped image is seen on the right.

In [ ]:
idx = numpy.random.randint(len(img_filenames))
img = imread(img_filenames[idx])
cropped_img = img[0:512, 0:512, :]
visualize(img, cropped_img)

**Task** : Visualize the bottom left portion of any  image

In [ ]:
# idx = numpy.random.randint(len(img_filenames))
# img = imread(img_filenames[idx])
# cropped_img = # TODO : fill correct value!!
# visualize(img, cropped_img)

## 2.2 Downsampling  <a class="anchor" name="eighth"></a>

For large images, sometimes we require that they are downsampled to fit in memory.

Say if one wants to have every fourth pixel from the original image, one specifies `factor` = $4$, and one can run the following cell:

In [ ]:
# downsampling
idx = numpy.random.randint(len(img_filenames))
img = imread(img_filenames[idx])

factor = 4
downsampled_img = img[::factor, ::factor]
visualize(img, downsampled_img)


**Task**: Can you see that the image on the right lacks some detail on account of being downsampled.

Try other values of the downsampling factors `factor`.

## 2.3 Flipping <a class="anchor" name="ninth"></a>

Sometimes, one wishes to create new images from original data by performing transformations.

One way to create a new image is by flipping an image about a given axis, which creates a mirror image!

Run the following cell to visualize a vertically flipped image.

In [ ]:
idx = numpy.random.randint(len(img_filenames))
img = imread(img_filenames[idx])
vflipped_img = img[::-1, :, :]
visualize(img, vflipped_img)

**Task** : Create a horizontally flipped image and visualize!

In [ ]:
#idx = numpy.random.randint(len(img_filenames))
#img = imread(img_filenames[idx])
#hflipped_img = ## TODO: fill correct value
#visualize(img, hflipped_img)

<hr>
Fantastic Work! 🙏

<h1>Checkpoint 2</h1>

In the second chapter, we learnt about:

<li> cropping images
<li> downsampling images
<li> flipping images

<hr>

**Bonus Task for Chapter 2**:

Can you think of reasons why we need to crop images?
Can't we feed in all the images at the original size to the model?

## 3.1 Creating Batches <a class="anchor" name="tenth"></a>

In ML/DL, we often have to deal with very large datasets. It is sometimes not possible to process all the data at once (since all of it wouldn't fit in the memory), so it's useful to split the data into "mini-batches".

Creating batches of data is also useful from the point of view of optimizing the model.

Let us make our first batch of images, containing $B$ number of images.
The shape of the batch will thus get an additional "batch dimension" at the first dimension, i.e. (batch_size, num_channels, height, width).

**Task**: Make a batch of size $B=4$ by sampling 4 images randomly from the available images (this will be a 4D numpy array).
<br> Here, you would also have to ensure that the second axis corresponds to the channel (use *numpy.transpose*) (See Task [1.4](#fourth)).





In [ ]:
#`batch` should be a numpy array with shape (4, 3, 1000, 1000).

# batch = # TODO
# print(batch.shape)

## 3.2 Convolutions <a class="anchor" name="eleventh"></a>

Convolutions are the elementary operations used in Convolutional Neural Networks (CNNs).

Below is a visual of the pixel values in the output image (green) being computed from neighboring pixels in the input image (blue) by convolving it with a filter (gray) which goes over the input image in a specific fashion.

![](https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/no_padding_no_strides.gif)

**Credit**: <a href="https://github.com/vdumoulin/conv_arithmetic">Vincent Dumoulin, Francesco Visin</a>


**Task** : Implement a function that performs a convolution of an image with a filter.

<br> Assume that your image and filter are square (i.e. height = width) and that the filter has an odd height/width. You can assume arbitrary values in your filter for now.

<br> Note that your output image will be smaller.


In [ ]:
from tqdm import tqdm
def conv2d(img, filter):
    assert filter.shape[0] == filter.shape[1]
    assert filter.shape[0]%2 !=0

    d = img.shape[0]  # height of original image
    d_f = filter.shape[0]  # height of filter
    # TODO
    return

**Task**: We noticed that the output image is smaller than the input image! <br>

Can you come up with an analytical relationship regarding how much smaller the output image is vis a vis the input image? <br>
Can you think of any strategy which ensures that the output image is the same size as the input image?

### 3.3 Design your own filter <a class="anchor" name="twelvth"></a>

Let us try to understand what the values of the filter should be. <br>


The following is known as the Sobel filter:

$$
\begin{bmatrix}
    1 & 2 & 1 \\
    0 & 0 & 0 \\
    -1 & -2 & -1
\end{bmatrix}
$$

**Task**: What pattern would produce the largest output when convolved with this filter? </b>

**Task**: Apply the Sobel filter and check if it does what you previously guessed! </b>


In [ ]:
filter = np.array([[1,2,1], [0, 0, 0], [-1, -2, -1]])
idx = numpy.random.randint(len(img_filenames))
img = imread(img_filenames[idx])
output_img = conv2d(img[..., 0], filter)
visualize(img[..., 0], output_img)


**Task**: Say if we are interested in finding roundish bright or dark structures in our data, what would be a reasonable filter? Discuss?



<hr>
Wow! 🤟

<h1>Checkpoint 3</h1>
In the third chapter, we learnt about:
<li> Creating batches </li>
<li> Convolutions </li>
<li> Designing your own filter</li>

<hr>

## (Optional) Data augmentation <a class="anchor" name="thirteenth"></a>

In ML/DL, we're often limited by the size of our the training set. Usually acquiring clean image per noisy image (image restoration); segmentation masks per raw image (image segmentation) etc is an arduous task.

How could we artificially increase our data to help the model generalize better?

One trick is to make simple transformations to our data - this process is generally called "data augmentation".  

`imgaug` is a Python library that provides a very extensive set of image augmentations. Let us import `imgaug` in the cell below.

In [ ]:
from imgaug import augmenters as iaa

#### Applying one augmentation

We can pick an augmentation from a list of available augmentations.

For example, with affine transformations, we can specify the range of the rotation angle to betwen `(-45, 45)` degrees.

In `imgaug`, the channel-axis is always expected to be the last axis and the batch - axis to be the first axis. <br> For that purpose, we artifically add a batch axis using `np.newaxis`.

In [ ]:
rotate = iaa.Affine(rotate=(-45, 45))
idx = numpy.random.randint(len(img_filenames))
img = imread(img_filenames[idx])
img_aug = rotate(images=img[np.newaxis, ...])
visualize(img, img_aug[0])

#### Applying multiple augmentations
We can linearly combine multiple augmentations.

In [ ]:
seq = iaa.Sequential([
    iaa.AdditiveGaussianNoise(scale=(0, 30)),
    iaa.pillike.FilterEdgeEnhanceMore(),
    iaa.Crop(percent=(0., 0.2)),
    rotate
])
img_aug = seq(images=img[np.newaxis, ...])
visualize(img, img_aug[0])

**Task**: Familiarize yourself with the different augmentations available through <code>imgaug</code>. <br>

Refer to the <a href = "https://github.com/aleju/imgaug">examples</a> and the <a href="https://imgaug.readthedocs.io/en/latest/">documentation</a>. <br>Try out and apply augmentations that you think are interesting.

**Task**: While augmenting images and segmentation masks, should they be augmented similarly or differently? Discuss.

<hr>

Hurrah! 😃
<h1>Checkpoint 4</h1>
In the fourth chapter, we learnt about:
<li> Using <code>imgaug</code> for augmenting images </li>
<li> Putting together multiple augmentations using <code>iaa.Sequential</code></li>
<hr>